In [ ]:
! conda install gdcm -c conda-forge -y

In [ ]:
# import some common libraries
import numpy as np
import os, json, cv2, random
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from PIL import Image
from tqdm.auto import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
INITIAL_TRAIN_PATH = '/kaggle/input/siim-covid19-detection/train'
INITIAL_TEST_PATH = '/kaggle/input/siim-covid19-detection/test'

TRAIN_PATH = '/tmp/train/'
TEST_PATH = '/tmp/test'

# Let's prepare images

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    image = apply_voi_lut(dicom.pixel_array, dicom)
    
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        image = np.amax(image) - image
        
    image = image - np.min(image)
    image = image / np.max(image)
    image = (image * 255).astype(np.uint8)
        
    return image

def dicom_to_jpg(path, path_to_save_to):
    for dirname, _, filenames in tqdm(os.walk(path), total=len(list(os.walk(path)))):

        for file in filenames:

            image = load_dicom(os.path.join(dirname, file))
            Image.fromarray(image).save(os.path.join(path_to_save_to, file.replace('dcm', 'jpg')))

In [ ]:
os.makedirs(TRAIN_PATH, exist_ok=True)
os.makedirs(TEST_PATH, exist_ok=True)
    
dicom_to_jpg(INITIAL_TRAIN_PATH, TRAIN_PATH)
dicom_to_jpg(INITIAL_TEST_PATH, TEST_PATH)

In [ ]:
! cd kaggle/working
! tar -czf train.tar.gz /tmp/train
! tar -czf test.tar.gz /tmp/test

In [ ]:
# from IPython.display import FileLink
# FileLink(r'./train.tar.gz')

In [ ]:
# FileLink(r'./test.tar.gz')